In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
sys.path.append("..")

In [11]:
from optimus import Optimus

In [12]:
# Create optimus
op = Optimus()

## Create dataframe
### Spark

This is ugly:

```
val someData = Seq(
  Row(8, "bat"),
  Row(64, "mouse"),
  Row(-27, "horse")
)

val someSchema = List(
  StructField("number", IntegerType, true),
  StructField("word", StringType, true)
)

val someDF = spark.createDataFrame(
  spark.sparkContext.parallelize(someData),
  StructType(someSchema)
)```

In [13]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True)

            ]
,
[
                ("  I like     fish  ", 1, "dog", "housé", "cat-car", "a","1"),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2"),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3"),
                (None, 3, "eagle", "glass", "lion-pc", "c","4")
            ])

df1 = op.create.df(
            [
         
                ("num 1", "int", True),
                ("animals 1", "str", True),
                ("thing 1", StringType(), True),
                ("filter 1", StringType(), True),
                ("num 2 1", "string", True)

            ]
,
[
                ( 1, "dog", "housé",  "a","1"),
                ( 2, "cat", "tv",  "b","2"),
                ( 2, "frog", "table","1","3"),
                ( 3, "eagle", "glass",  "c","4")
            ])

df2 = op.create.df(
            [
         
                ("num 2", "int", True),
                ("filter 2", StringType(), True),
        

            ]
,
[
                ( 1, "a"),
                ( 2,  "b"),
                ( 2, "1"),
                ( 3, "c")
            ])

df.show()
df1.show()
df2.show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+

+-----+---------+-------+--------+-------+
|num 1|animals 1|thing 1|filter 1|num 2 1|
+-----+---------+-------+--------+-------+
|    1|      dog|  housé|       a|      1|
|    2|      cat|     tv|       b|      2|
|    2|     frog|  table|       1|      3|
|    3|    eagle|  glass|       c|      4|
+-----+---------+-------+--------+-------+

+-----+--------+
|num 2|filter 2|
+-----+--------+
|    1|       a|
|    2|       b|
|    2|       1|
|    3|       c|

## Create Columns
### Spark
* You can not create multiple columns at the same time
* You need to use the lit function. lit???

### Pandas
* Assing function seems to do the job https://stackoverflow.com/questions/12555323/adding-new-column-to-existing-dataframe-in-python-pandas


In [14]:
df.cols().append("new_col_1", (1,2)).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|   [1, 2]|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|   [1, 2]|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|   [1, 2]|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|   [1, 2]|
+-------------------+---+-------+-----+-------------+------+-----+---------+



In [15]:
from pyspark.sql.functions import *

df.cols().append([
    ("float", 2.22),
    ("integer", lit(3)),
    ("string", "test"),
    ("boolean", True),       
    ("multi", df['num']*2),
    ("array", [1,2,3]),
    ("tuple", (6,7,8))
    
    ]).show()



+-------------------+---+-------+-----+-------------+------+-----+-----+-------+------+-------+-----+---------+---------+
|              words|num|animals|thing|  two strings|filter|num 2|float|integer|string|boolean|multi|    array|    tuple|
+-------------------+---+-------+-----+-------------+------+-----+-----+-------+------+-------+-----+---------+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1| 2.22|      3|  test|   true|    2|[1, 2, 3]|[6, 7, 8]|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2| 2.22|      3|  test|   true|    4|[1, 2, 3]|[6, 7, 8]|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3| 2.22|      3|  test|   true|    4|[1, 2, 3]|[6, 7, 8]|
|               null|  3|  eagle|glass|      lion-pc|     c|    4| 2.22|      3|  test|   true|    6|[1, 2, 3]|[6, 7, 8]|
+-------------------+---+-------+-----+-------------+------+-----+-----+-------+------+-------+-----+---------+---------+



In [16]:
df.cols().append([df1,df2]).show()

+-------------------+---+-------+-----+-------------+------+-----+-----+---------+-------+--------+-------+-----+--------+
|              words|num|animals|thing|  two strings|filter|num 2|num 1|animals 1|thing 1|filter 1|num 2 1|num 2|filter 2|
+-------------------+---+-------+-----+-------------+------+-----+-----+---------+-------+--------+-------+-----+--------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|    1|      dog|  housé|       a|      1|    1|       a|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|    2|      cat|     tv|       b|      2|    2|       b|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|    3|    eagle|  glass|       c|      4|    3|       c|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|    2|     frog|  table|       1|      3|    2|       1|
+-------------------+---+-------+-----+-------------+------+-----+-----+---------+-------+--------+-------+-----+--------+



## Select columns
### Spark
* You can not select columns by string and index at the same time

### Pandas
* You can not select columns by string and index at the same time

In [17]:
columns = ["words", 1, "animals", 3, 0]
df.cols().filter(columns).show()

+-------------------+---+-------+-----+-------------------+
|              words|num|animals|thing|              words|
+-------------------+---+-------+-----+-------------------+
|  I like     fish  |  1|    dog|housé|  I like     fish  |
|            zombies|  2|    cat|   tv|            zombies|
|simpsons   cat lady|  2|   frog|table|simpsons   cat lady|
|               null|  3|  eagle|glass|               null|
+-------------------+---+-------+-----+-------------------+



In [18]:
df.cols().filter("n.*", regex = True).show()

+---+-----+
|num|num 2|
+---+-----+
|  1|    1|
|  2|    2|
|  2|    3|
|  3|    4|
+---+-----+



In [22]:
df.cols().filter("*", data_type = "int").show()

+---+
|num|
+---+
|  1|
|  2|
|  2|
|  3|
+---+



## Rename Column
### Spark
You can not rename multiple columns using Spark Vanilla API


### Pandas
* Almost the same behavior https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html

In [40]:
df.cols().rename([('num','number')]).show()

+-------------------+------+-------+-----+-------------+------+-----+
|              words|number|animals|thing|  two strings|filter|num 2|
+-------------------+------+-------+-----+-------------+------+-----+
|  I like     fish  |     1|    dog|housé|      cat-car|     a|    1|
|            zombies|     2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|     2|   frog|table|eagle-tv-plus|     1|    3|
|               null|     3|  eagle|glass|      lion-pc|     c|    4|
+-------------------+------+-------+-----+-------------+------+-----+



In [22]:
df.cols().rename(func = str.lower).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [23]:
df.cols().rename(func = str.upper).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              WORDS|NUM|ANIMALS|THING|  TWO STRINGS|FILTER|NUM 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



## Cast a columns

### Spark
* Can not cast multiple columns

### Pandas
This is a opinionated way to handle column casting. 
One of the first thing that every data cleaning process need to acomplish is define a data dictionary.
Because of that we prefer to create a tuple like this:

df.cols().cast(
[("words","str"),
("num","int"),
("animals","float"),
("thing","str")]
)


https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.astype.html

In [25]:
df.cols().cast([("num", "string"),("num 2", "integer")])

DataFrame[words: string, num: string, animals: string, thing: string, two strings: string, filter: string, num 2: int]

## Keep columns
### Spark
* You can not remove multiple columns

### Pandas
* Handle in pandas with drop


In [26]:
from pyspark.sql.functions import *
df.withColumn("num", col("num").cast(StringType()))


DataFrame[words: string, num: string, animals: string, thing: string, two strings: string, filter: string, num 2: string]

In [27]:
df.cols().keep("num").show()

+---+
|num|
+---+
|  1|
|  2|
|  2|
|  3|
+---+



## Move columns
### Spark
Do not exist in spark

### Pandas
Do not exist in pandas

In [28]:
df.cols().move("words", "thing", "after").show()

+---+-------+-----+-------------------+-------------+------+-----+
|num|animals|thing|              words|  two strings|filter|num 2|
+---+-------+-----+-------------------+-------------+------+-----+
|  1|    dog|housé|  I like     fish  |      cat-car|     a|    1|
|  2|    cat|   tv|            zombies|       dog-tv|     b|    2|
|  2|   frog|table|simpsons   cat lady|eagle-tv-plus|     1|    3|
|  3|  eagle|glass|               null|      lion-pc|     c|    4|
+---+-------+-----+-------------------+-------------+------+-----+



## Sorting Columns
### Spark
You can not sort columns using Spark Vanilla API 

### Pandas
Similar to pandas
http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.DataFrame.sort_values.html#pandas.DataFrame.sort_values

In [33]:
df.cols().sort().show()

+-------+------+---+-----+-----+-------------+-------------------+
|animals|filter|num|num 2|thing|  two strings|              words|
+-------+------+---+-----+-----+-------------+-------------------+
|    dog|     a|  1|    1|housé|      cat-car|  I like     fish  |
|    cat|     b|  2|    2|   tv|       dog-tv|            zombies|
|   frog|     1|  2|    3|table|eagle-tv-plus|simpsons   cat lady|
|  eagle|     c|  3|    4|glass|      lion-pc|               null|
+-------+------+---+-----+-----+-------------+-------------------+



In [36]:
df.cols().sort(order = "desc").show()

+-------------------+-------------+-----+-----+---+------+-------+
|              words|  two strings|thing|num 2|num|filter|animals|
+-------------------+-------------+-----+-----+---+------+-------+
|  I like     fish  |      cat-car|housé|    1|  1|     a|    dog|
|            zombies|       dog-tv|   tv|    2|  2|     b|    cat|
|simpsons   cat lady|eagle-tv-plus|table|    3|  2|     1|   frog|
|               null|      lion-pc|glass|    4|  3|     c|  eagle|
+-------------------+-------------+-----+-----+---+------+-------+



## Drop columns
### Spark 
* You can not delete multiple colums

### Pandas
* Almost the same as pandas
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [35]:
df2 = df.cols().drop("num")
df2 = df.cols().drop(["num","words"])
df2.show()

+-------+-----+-------------+------+-----+
|animals|thing|  two strings|filter|num 2|
+-------+-----+-------------+------+-----+
|    dog|housé|      cat-car|     a|    1|
|    cat|   tv|       dog-tv|     b|    2|
|   frog|table|eagle-tv-plus|     1|    3|
|  eagle|glass|      lion-pc|     c|    4|
+-------+-----+-------------+------+-----+



## Chaining

cols y rows functions are used to organize and encapsulate optimus' functionality apart of Apache Spark Dataframe API. This have a disadvantage at chaining time because we need to user invoke cols or rows in every step.

At the same time it can be helpfull when you look at the code because every line is self explained.

In [23]:
df.show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [30]:
from pyspark.sql import functions  as F

df\
    .cols().rename([("num","number")])\
    .cols().drop(["number","words"])\
    .withColumn("new_col_2", lit("spongebob"))\
    .cols().append("new_col_1", 1)\
    .rows().drop(F.col("filter")=="1")\
    .cols().sort(order= "desc")\
    .cols().drop("two strings")\
    .show()

+-----+-----+---------+---------+------+-------+
|thing|num 2|new_col_2|new_col_1|filter|animals|
+-----+-----+---------+---------+------+-------+
|housé|    1|spongebob|        1|     a|    dog|
|   tv|    2|spongebob|        1|     b|    cat|
|glass|    4|spongebob|        1|     c|  eagle|
+-----+-----+---------+---------+------+-------+



In [101]:
from pyspark.sql.functions import *

def func(value, args):
    return value + 10
    
df\
    .cols().rename([("num","number")])\
    .withColumn("new_col_2", lit("spongebob"))\
    .cols().apply("number", func, "int")\
    .cols().apply_exp("new_col_2", F.col("number")+1)\
    .cols().drop("words")\
    .show()

+------+-------+-----+-------------+------+-----+---------+
|number|animals|thing|  two strings|filter|num 2|new_col_2|
+------+-------+-----+-------------+------+-----+---------+
|    11|    dog|housé|      cat-car|     a|    1|       12|
|    12|    cat|   tv|       dog-tv|     b|    2|       13|
|    12|   frog|table|eagle-tv-plus|     1|    3|       13|
|    13|  eagle|glass|      lion-pc|     c|    4|       14|
+------+-------+-----+-------------+------+-----+---------+



## Split Columns
### Spark

### Pandas

In [21]:
df.cols().split("two strings","-", n=3).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+-----+-----+-----+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|COL_0|COL_1|COL_2|
+-------------------+---+-------+-----+-------------+------+-----+---------+-----+-----+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|  cat|  car| null|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|        1|  dog|   tv| null|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|        1|eagle|   tv| plus|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|        1| lion|   pc| null|
+-------------------+---+-------+-----+-------------+------+-----+---------+-----+-----+-----+



In [22]:
df.cols().split("two strings","-", get = 1).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|COL_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+-----+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|  car|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|        1|   tv|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|        1|   tv|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|        1|   pc|
+-------------------+---+-------+-----+-------------+------+-----+---------+-----+



## Impute
### Spark

In [23]:
df_cast =df.cols().cast([("num","double"),("num 2", "double")])
df_cast.dtypes

df_cast.cols().impute(["num","num 2"], ["out_a","out_B"], strategy="mean")
df_cast.show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |1.0|    dog|housé|      cat-car|     a|  1.0|        1|
|            zombies|2.0|    cat|   tv|       dog-tv|     b|  2.0|        1|
|simpsons   cat lady|2.0|   frog|table|eagle-tv-plus|     1|  3.0|        1|
|               null|3.0|  eagle|glass|      lion-pc|     c|  4.0|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+



## Get columns by type
### Spark

### Pandas

In [24]:
df.cols().filter_by_dtypes("int").show()

+---+---------+
|num|new_col_1|
+---+---------+
|  1|        1|
|  2|        1|
|  2|        1|
|  3|        1|
+---+---------+



## Apply custom function
### Spark
You need to declare a UDF Spark function

### Pandas
Almost the same behavior

In [25]:
df.show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|        1|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|        1|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+



In [26]:
def sum_(val, attr):
    return attr

df.cols().apply_by_dtypes("filter", sum_, "string", "10", filter_by_data_type="integer").show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|        1|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|    10|    3|        1|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+



In [7]:
def func(val, attr):
    return val + attr

df.cols().apply(["num", "num"], func, "int", 32 ,"udf").show()

32
<function func at 0x0000016C72BFAA60>
32
<function func at 0x0000016C72BFAA60>
+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  | 65|    dog|housé|      cat-car|     a|    1|
|            zombies| 66|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady| 66|   frog|table|eagle-tv-plus|     1|    3|
|               null| 67|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [99]:
def func(val, attr):
    return val + attr

df.cols().apply(["num", "num"], func, "int", 32 ,"udf").show()

32
32
+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  | 65|    dog|housé|      cat-car|     a|    1|
|            zombies| 66|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady| 66|   frog|table|eagle-tv-plus|     1|    3|
|               null| 67|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [1]:
from optimus.functions import filter_row_by_data_type as fbdt

df.where(fbdt("filter", "integer")).show()

ModuleNotFoundError: No module named 'optimus'

In [30]:
def func(val, attr):
    return val + attr

df.cols().apply(["num", "num"], func, "int", 10).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  | 21|    dog|housé|      cat-car|     a|    1|        1|
|            zombies| 22|    cat|   tv|       dog-tv|     b|    2|        1|
|simpsons   cat lady| 22|   frog|table|eagle-tv-plus|     1|    3|        1|
|               null| 23|  eagle|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+



## Abstract UDF

In [33]:
from optimus.functions import abstract_udf as audf 

def func(val, attr):
    return val>1

df.rows().filter(audf("num", func, "bool")).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [35]:
from optimus.functions import abstract_udf as audf 
from optimus.functions import filter_row_by_data_type as frdt
def func(val, attr):
    return val>2

df.rows().filter(audf("num", func, "bool")).show()

+-----+---+-------+-----+-----------+------+-----+
|words|num|animals|thing|two strings|filter|num 2|
+-----+---+-------+-----+-----------+------+-----+
| null|  3|  eagle|glass|    lion-pc|     c|    4|
+-----+---+-------+-----+-----------+------+-----+



In [32]:
# https://stackoverflow.com/questions/31400143/column-filtering-in-pyspark
from optimus.functions import abstract_udf as audf 

def func(val, attr):
    return val+attr[0]+attr[1]


df.withColumn("num", audf ("num", func, "int", [10,20])).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  | 31|    dog|housé|      cat-car|     a|    1|
|            zombies| 32|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady| 32|   frog|table|eagle-tv-plus|     1|    3|
|               null| 33|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [107]:
def func(col_name, attr):
    return F.col(col_name) + 1

df.withColumn("num", audf ("num", func, "int", ["num","num"], "column_exp")).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  2|    dog|housé|      cat-car|     a|    1|
|            zombies|  3|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  3|   frog|table|eagle-tv-plus|     1|    3|
|               null|  4|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



## Apply Exp

In [41]:
from pyspark.sql import functions as F
def func(col_name, attr):
    # return F.col(col_name) + 1
    return F.when(F.col(col_name)>0 ,2)

df.cols().apply_exp(["num","num 2"], func).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  2|    dog|housé|      cat-car|     a|    2|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    2|
|               null|  2|  eagle|glass|      lion-pc|     c|    2|
+-------------------+---+-------+-----+-------------+------+-----+



In [42]:
df.cols().apply_exp(["num"], F.when(F.col("num")>0 ,2)).show()

+-------------------+---+-------+-----+-------------+------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|
+-------------------+---+-------+-----+-------------+------+-----+
|  I like     fish  |  2|    dog|housé|      cat-car|     a|    1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|
|               null|  2|  eagle|glass|      lion-pc|     c|    4|
+-------------------+---+-------+-----+-------------+------+-----+



In [33]:
from pyspark.sql import functions as F
def func(col_name, attr):
    return F.upper(F.col(col_name))

df.cols().apply_exp(["two strings","animals"], func).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    DOG|housé|      CAT-CAR|     a|    1|        1|
|            zombies|  2|    CAT|   tv|       DOG-TV|     b|    2|        1|
|simpsons   cat lady|  2|   FROG|table|EAGLE-TV-PLUS|     1|    3|        1|
|               null|  3|  EAGLE|glass|      LION-PC|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+



In [34]:
def func(val, attr):
    return 10

col = "num"

df.cols().apply(col, func, "int", when= df["num"]>1).show()

df.cols().apply(col, func, "int", when= fbdt(col, "integer")).show()


+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|
|            zombies| 10|    cat|   tv|       dog-tv|     b|    2|        1|
|simpsons   cat lady| 10|   frog|table|eagle-tv-plus|     1|    3|        1|
|               null| 10|  eagle|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  | 10|    dog|housé|      cat-car|     a|    1|        1|
|            zombies| 10|    cat|   tv|       dog-tv|     b|    2|        1

## Count Nulls
### Spark

### Pandas

In [35]:
import numpy as np

df_null = op.get_ss().createDataFrame(
    [(1, 1, None), (1, 2, float(5)), (1, 3, np.nan), (1, 4, None), (1, 5, float(10)), (1, 6, float('nan')), (1, 6, float('nan'))],
    ('session', "timestamp1", "id2"))

In [36]:
df_null.cols().count_na("id2")

{'id2': 5}

In [37]:
df_null.cols().count_na("*")

{'id2': 5, 'session': 0, 'timestamp1': 0}

## Count uniques
### Spark

### Pandas


In [38]:
df.cols().count_uniques("*")

{'animals': 4,
 'filter': 4,
 'new_col_1': 1,
 'num': 3,
 'num 2': 4,
 'thing': 4,
 'two strings': 4,
 'words': 4}

## Unique
### Spark
An abstraction of distinct to be use in multiple columns at the same time

### Pandas
Similar behavior than pandas

In [39]:
df.show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|        1|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|        1|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+



In [40]:
df_distinct = op.create.df(
            [
                ("words", "str", True),
                ("num", "int", True)
            ],
[
                ("  I like     fish  ", 1),
                ("    zombies", 2),
                ("simpsons   cat lady", 2),
                (None, 3),
                  (None, 0)
            ])

In [41]:
df_distinct\
    .select("num")\
    .cols().unique().show()

+---+
|num|
+---+
|  1|
|  3|
|  2|
|  0|
+---+



## Count Zeros

In [42]:
df_zeros = df_distinct
df_zeros.show()
df_zeros.cols().count_zeros("*")

+-------------------+---+
|              words|num|
+-------------------+---+
|  I like     fish  |  1|
|            zombies|  2|
|simpsons   cat lady|  2|
|               null|  3|
|               null|  0|
+-------------------+---+



{'num': 1, 'words': 0}

## Column Data Types

In [43]:
df.cols().dtypes('*')

{'animals': 'string',
 'filter': 'string',
 'new_col_1': 'int',
 'num': 'int',
 'num 2': 'string',
 'thing': 'string',
 'two strings': 'string',
 'words': 'string'}

## Replace

In [44]:
df.cols().replace(["two strings","animals"], ["dog-tv", "cat", "eagle", "fish"], "animals").show()

df.cols().replace('animals',["dog","cat"],"animals").show()

df.cols().replace('animals',[("dog","animals"),("cat","animals")]).show()

df.cols().replace('animals',"dog",'animal').show()

df.cols().replace('num',["3",2], 10).show()

df.cols().replace('num',[("3",6),(2,6)]).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|      cat-car|     a|    1|        1|
|            zombies|  2|animals|   tv|      animals|     b|    2|        1|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|        1|
|               null|  3|animals|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|animals|housé|      cat-car|     a|    1|        1|
|            zombies|  2|animals|   tv|       dog-tv|     b|    2|        1

In [45]:
df.cols().replace('*','.*[Cc]at.*', 'animal', regex=True).show()
df.cols().replace('*','cat', 'animal', regex=True).show()

+-------------------+---+-------+-----+-------------+------+-----+---------+
|              words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+-------------------+---+-------+-----+-------------+------+-----+---------+
|  I like     fish  |  1|    dog|housé|       animal|     a|    1|        1|
|            zombies|  2| animal|   tv|       dog-tv|     b|    2|        1|
|             animal|  2|   frog|table|eagle-tv-plus|     1|    3|        1|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|        1|
+-------------------+---+-------+-----+-------------+------+-----+---------+

+--------------------+---+-------+-----+-------------+------+-----+---------+
|               words|num|animals|thing|  two strings|filter|num 2|new_col_1|
+--------------------+---+-------+-----+-------------+------+-----+---------+
|   I like     fish  |  1|    dog|housé|   animal-car|     a|    1|        1|
|             zombies|  2| animal|   tv|       dog-tv|     b|    2|    